# Evaluation code for filter_dp

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import opendp as dp

Load and clean dataset

In [ ]:
# Load input datasets
input_salaries = pd.read_csv('input/EmployeeSalaries.csv', names=["Department","Department_Name","Division","Gender","Base_Salary","Overtime_Pay","Longevity_Pay","Grade"])
input_students = pd.read_csv('input/StudentsPerformance.csv', names=["gender","race_ethnicity","parental_education","lunch","test_preparation","math_score","reading_score","writing_score"])

# Load output datasets
output_salaries = pd.read_csv('output/EmployeeSalaries.perturbed.csv', names=["Department","Department_Name","Division","Gender","Base_Salary","Overtime_Pay","Longevity_Pay","Grade"])
output_students = pd.read_csv('output/StudentsPerformance.perturbed.csv', names=["gender","race_ethnicity","parental_education","lunch","test_preparation","math_score","reading_score","writing_score"])

# Remove the last line from the "student" input/output files
input_students.drop(input_students.tail(1).index,inplace=True)
output_students.drop(output_students.tail(1).index,inplace=True)

In [ ]:
# Salaries dataset analysis
print("Salaries Dataset:")
print("Original Base_Salary - Mean: {:.2f}, Std: {:.2f}".format(input_salaries['Base_Salary'].mean(), input_salaries['Base_Salary'].std()))
print("Perturbed Base_Salary - Mean: {:.2f}, Std: {:.2f}".format(output_salaries['Base_Salary'].mean(), output_salaries['Base_Salary'].std()))

plt.figure(figsize=(8, 6))
plt.hist(input_salaries['Base_Salary'], bins=250, alpha=0.5, label='Original')
plt.hist(output_salaries['Base_Salary'], bins=250, alpha=0.5, label='Perturbed')
plt.xlabel('Base_Salary')
plt.ylabel('Frequency')
plt.legend()
plt.show()

# Students dataset analysis
print("\nStudents Dataset:")
for score in ['math_score', 'reading_score', 'writing_score']:
    print("Original {} - Mean: {:.2f}, Std: {:.2f}".format(score, input_students[score].mean(), input_students[score].std()))
    print("Perturbed {} - Mean: {:.2f}, Std: {:.2f}".format(score, output_students[score].mean(), output_students[score].std()))

plt.figure(figsize=(8, 6))
for score in ['math_score', 'reading_score', 'writing_score']:
    plt.hist(input_students[score], bins=100, alpha=0.5, label='Original ' + score)
    plt.hist(output_students[score], bins=100, alpha=0.5, label='Perturbed ' + score)
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()